# Setup: Connectivity

In [1]:

from katana import remote
   #
from IPython.display import display
from ipywidgets import Layout, Textarea, VBox
   #
from libraries.SupportFor34 import f_init, f_enrich
    
    
my_client, my_graph = f_init()


print("My Client: %s" % (my_client))
print("My Graph:  %s" % (my_graph ))
   #
print()


My Client: <katana_enterprise.remote.sync_wrappers.Client object at 0x7ffb603b4280>
My Graph:  <_Graph my_graph, D2zAjhTGfG7UKNLjAh48GNwcNCSFbGC2L38v3Rv2LK2M, 5>



#  Step 01: Enter a new Medical Description

In [2]:

#  Create a text entry area
#

l_layout   = Layout(flex = "0 1 auto", height = "100px", min_height = "100px", width = "auto")
l_text01   = Textarea(value="Odd mole on my ear", layout = l_layout, description = "Enter Text: ")
   #
l_vbox     = VBox([l_text01], layout = Layout(flex = "1 1 auto", width = "auto"))
display(l_text01)


Textarea(value='Odd mole on my ear', description='Enter Text: ', layout=Layout(flex='0 1 auto', height='100px'…

In [3]:

#  Enrich/decode text entry from above
#

l_return, l_pvid = f_enrich(l_text01.value.lower(), my_graph)

print("Unique Patient Visit Id: %s" % (l_pvid))
print("")
print("Medical Description encoding:")
print("")
display(l_return)


          0/? [?op/s]

Unique Patient Visit Id: PV-2033

Medical Description encoding:



{'entityMentions': [{'mentionId': '1',
   'type': 'PROBLEM',
   'text': {'content': 'mole', 'beginOffset': 4},
   'linkedEntities': [{'entityId': 'UMLS/C0027960'},
    {'entityId': 'UMLS/C0027962'}],
   'temporalAssessment': {'value': 'UPCOMING',
    'confidence': 0.6084825992584229},
   'certaintyAssessment': {'value': 'LIKELY',
    'confidence': 0.057481713593006134},
   'subject': {'value': 'PATIENT', 'confidence': 0.9589220881462097},
   'confidence': 0.5596652626991272},
  {'mentionId': '2',
   'type': 'ANATOMICAL_STRUCTURE',
   'text': {'content': 'ear', 'beginOffset': 15},
   'linkedEntities': [{'entityId': 'UMLS/C0013443'}],
   'confidence': 0.9025713205337524}],
 'entities': [{'entityId': 'UMLS/C0013443',
   'preferredTerm': 'Ear structure',
   'vocabularyCodes': ['FMA/52780',
    'FMA/77739',
    'LNC/LA21929-7',
    'LNC/LA22163-2',
    'LNC/LP7188-8',
    'LNC/MTHU001407',
    'MSH/D004423',
    'MTH/NOCODE',
    'NCI/C12394',
    'OMIM/MTHU000052']},
  {'entityId': 'UMLS/C

# Step 02:  Examine the Graph

In [4]:

#  Produce a visual graph

from katana_visualization_widget import GraphVisOptions, NodeVisOption, EdgeVisOption, ANY

l_options = GraphVisOptions(
   node_options = [
      NodeVisOption("PatientVisit",     label="short_text"       ),
      NodeVisOption("UmlsEntity",       label="id"               ),
      NodeVisOption("UmlsVocabulary",   label="id"               ),
   ],
   #  edge_options = [
   #     EdgeVisOption(["VISIT_CONTAINS"], label="start_id"),
   #     EdgeVisOption(["ALSO_CODED_AS" ], label="start_id"),
   #  ]
   )
    
    
l_query  = """

   MATCH (pv: PatientVisit) - [vc: VISIT_CONTAINS] -> (ue: UmlsEntity) - [aca: ALSO_CODED_AS] -> (uv: UmlsVocabulary)
   WHERE
      (pv.id = '{0}')
   RETURN pv, vc, ue, aca, uv
   
   """.format(l_pvid)

l_result = my_graph.query_unpaginated(l_query, contextualize=True)
    
l_result.view(graph_vis_options = l_options)


          0/? [?op/s]

          0/? [?op/s]

/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


In [ ]:
# #########################################################################

In [5]:

#  Checking just nodes
#

l_query  = """

   MATCH (n)
   WHERE NOT n.id STARTS WITH 'XX-'                   //  A value we use as a placeholder
   RETURN n
   LIMIT 100
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)



          0/? [?op/s]

    n.internal_id          n.labels         n.LABEL           n.id n.vocabulary_code n.type    n.entity_id                        n.preferred_term
0         2009375  [UmlsVocabulary]  UmlsVocabulary    NCBI/287187       NCBI/287187   node            NaN                                     NaN
1         2009376  [UmlsVocabulary]  UmlsVocabulary   NCBI/2871891      NCBI/2871891   node            NaN                                     NaN
2         2009377  [UmlsVocabulary]  UmlsVocabulary   NCBI/2871892      NCBI/2871892   node            NaN                                     NaN
3         2009378  [UmlsVocabulary]  UmlsVocabulary   NCBI/2871894      NCBI/2871894   node            NaN                                     NaN
4         2009379  [UmlsVocabulary]  UmlsVocabulary   NCBI/2871899      NCBI/2871899   node            NaN                                     NaN
..            ...               ...             ...            ...               ...    ...            ...            

/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


In [ ]:

#  Checking just edges
#

l_query  = """

   MATCH (n) - [r] -> (m)
   RETURN r
   LIMIT 100
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)


In [ ]:
# #########################################################################

In [ ]:

#  Metadata on our Nodes,
#
#  Sample output, smaller load,
#
#     +----+------------------+-------+
#     |    | label            |   cnt |
#     |----+------------------+-------|
#     |  0 | Keyword          | 10443 |
#     |  1 | MedicalSpecialty |    40 |
#     |  2 | PatientVisit     |  4999 |
#     |  3 | UmlsEntity       | 20073 |
#     |  4 | UmlsVocabulary   | 40148 |
#     +----+------------------+-------+

#  Sample output, after the larger load,
#
#     +----+------------------+---------+
#     |    | label            |     cnt |
#     |----+------------------+---------|
#     |  0 | Keyword          |   10443 |
#     |  1 | MedicalSpecialty |      40 |
#     |  2 | PatientVisit     |    4999 |
#     |  3 | UmlsEntity       | 4661972 |
#     |  4 | UmlsVocabulary   | 3365900 |
#     +----+------------------+---------+


In [6]:

#  Checking just nodes
#

l_query  = """

   MATCH (n: UmlsEntity)
   RETURN n
   LIMIT 20
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)



          0/? [?op/s]

    n.internal_id      n.labels     n.LABEL    n.entity_id           n.id                                   n.preferred_term n.type
0            6304  [UmlsEntity]  UmlsEntity  UMLS/C0223098  UMLS/C0223098             Structure of body of cervical vertebra   node
1            6305  [UmlsEntity]  UmlsEntity  UMLS/C0225219  UMLS/C0225219  Structure of prevertebral layer of cervical fa...   node
2            6306  [UmlsEntity]  UmlsEntity  UMLS/C0229427  UMLS/C0229427                             Pneumatic mastoid cell   node
3            6307  [UmlsEntity]  UmlsEntity  UMLS/C0501396  UMLS/C0501396                                Collateral ganglion   node
4            6308  [UmlsEntity]  UmlsEntity  UMLS/C4476754  UMLS/C4476754    Decreased attenuation pattern on pulmonary HRCT   node
5            6309  [UmlsEntity]  UmlsEntity  UMLS/C4520924  UMLS/C4520924                                 Bone Tissue, Human   node
6            6310  [UmlsEntity]  UmlsEntity  UMLS/C0026608  UMLS/C0026608   

/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response
